# OpenAI Experimentation

The purpose of this notebook is to try out the different OpenAI features to familiarise myself with them, with the ultimate aim of using GPT-3 for Named Entity Recognition to anonymise our care notes reliably for analytics.

The features tested will be:
- Completion: Training GPT-3 to response to prompts to perform useful tasks

https://beta.openai.com/docs/guides/completion/introduction

## Setup

This step contains any imports and common setup functions

In [2]:
import os
import openai

from helpers import OPENAI_API_KEY

In [3]:
openai.api_key = OPENAI_API_KEY

In [4]:
openai.Engine.list()

<OpenAIObject list at 0x11624af50> JSON: {
  "data": [
    {
      "created": null,
      "id": "ada",
      "max_replicas": null,
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true,
      "replicas": null
    },
    {
      "created": null,
      "id": "ada-code-search-code",
      "max_replicas": null,
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true,
      "replicas": null
    },
    {
      "created": null,
      "id": "ada-code-search-text",
      "max_replicas": null,
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true,
      "replicas": null
    },
    {
      "created": null,
      "id": "ada-instruct-beta",
      "max_replicas": null,
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true,
      "replicas": null
    },
    {
      "created": null,
      "id": "ada-search-document",
      "max_re

## Named Entity Recognition

We want to remove personal names from this text

In [7]:
care_notes = '''

    The care notes are:
    
    Alice is well in.herself this evening, made a cup tea on.my arrival and 
    put a frozen meal in the microwave and heated go manufacturers guidelines, 
    tonight's meal had too much rice. 
    Alice didn't enjoy all of the rice,washed up chatted and leaving Alice comfy with her cup of tea.
    
    Ahmed was also there. he is often there as he is her neighbour, and she calls him ahmy. Ahmy was helpful. We then left
    
    Remove all personal names from the care notes and replace with Customer
'''

In [8]:
anonymised_care_notes = openai.Completion.create(engine="davinci-instruct-beta-v3", prompt=care_notes, max_tokens=len(care_notes) + 50)

In [9]:
print(anonymised_care_notes['choices'][0]['text'])


The care notes are:

Customer is well in herself this evening, made a cup tea on my arrival and put a frozen meal in the microwave and heated it to manufacturers guidelines, tonight's meal had too much rice. Customer didn't enjoy all of the rice, washed up chatted and leaving customer comfy with her cup of tea. Ahmed was also there. he is often there as he is her neighbour, and she calls him ahmy. Ahmy was helpful. We then left.


In [26]:
care_notes_updated = '''

    The care notes are:
    
    Alice is well in.herself this evening, made a cup tea on.my arrival and 
    put a frozen meal in the microwave and heated go manufacturers guidelines, 
    tonight's meal had too much rice. Alice didn't enjoy all of the rice,washed up chatted and leaving Alice comfy with her cup of tea.
    Ahmed was also there. he is often there as he is her neighbour, and she calls him ahmy. Ahmy was helpful. We then left
    
    Instructions:
    
    Remove all personal names from the care notes and replace with Customer
    Replace all pronouns with they/them
'''

In [27]:
anonymised_better_care_notes = openai.Completion.create(engine="davinci-instruct-beta-v3", prompt=care_notes_updated, max_tokens=len(care_notes_updated) + 50)

In [28]:
print(anonymised_better_care_notes['choices'][0]['text'])


The care notes are:

The customer is well in themselves this evening. They made a cup of tea on my arrival and heated a frozen meal according to manufacturer guidelines. Tonight's meal had too much rice. They didn't enjoy all of the rice, but they cleaned up and chatted. We then left.

Instructions:

Remove all personal names from the care notes and replace with customer.
Replace all pronouns with they/them.


### Structured Input Prompt

Let's try giving it more explicit instructions

In [39]:
structured_input_notes = '''
    You are anonymising care notes from social care appointments.
    
    Anonymising care notes requires:
    - All names to be removed and replaced with Customer
    - All pronouns to be replaced with they/them
    - Nicknames and names from other languages should be removed and replaced with Customer
    
    The care notes are:
    
    Alice is well in herself this evening, made a cup tea on.my arrival and 
    put a frozen meal in the microwave and heated go manufacturers guidelines, 
    tonight's meal had too much rice. Alice didn't enjoy all of the rice,washed up chatted and leaving Alice comfy with her cup of tea. Ahmed was also there. he is often there as he is her neighbour, and she calls him ahmy. Ahmy was helpful. We then left.
    
    The anonymised care notes are:
    
'''

In [9]:
prompt_guide = '''You are anonymising care notes from social care appointments.
    
    Anonymising care notes requires:
    - All names to be removed and replaced with Customer
    - All pronouns to be replaced with they/them
    - Nicknames and names from other languages should be removed and replaced with Customer
    
    The care notes are: '''
care_notes = ['''Alice is well in herself this evening, made a cup tea on.my arrival and 
    put a frozen meal in the microwave and heated go manufacturers guidelines, 
    tonight's meal had too much rice. Alice didn't enjoy all of the rice,washed up chatted and leaving Alice comfy with her cup of tea.'''
             ,'''Came and visited Cam, he seemed a bit down. Changed the bed, gave medication. Didn't give a drink, Cameron said he had drank before I came. All well when I left'''
             ,'''Tracy was aggressive and pushed me over. Raised an incident form and reported to the office, then completed my tasks. She calmed down and told me she was sorry, then I left''']
closer = 'The anonymised care notes are:'

In [10]:
for note in care_notes:
    answered_notes = openai.Completion.create(engine="davinci-instruct-beta-v3", prompt=prompt_guide+note+closer, max_tokens=len(note)+50)
    print(answered_notes['choices'][0]['text'])

 Customer is well in themselves this evening, made a cup tea on my arrival and put a frozen meal in the microwave and heated according to manufacturer guidelines, tonight's meal had too much rice. Customer didn't enjoy all of the rice, washed up chatted and left Customer comfy with their cup of tea
 Came and visited Customer, they seemed a bit down. Changed the bed, gave medication. Didn't give a drink, Customer said they had drank before I came. All well when I left
 Customer was aggressive and pushed me over. Raised an incident form and reported to the office, then completed my tasks. They calmed down and told me they were sorry, then I left


In [40]:
anonymised_structured_care_notes = openai.Completion.create(engine="davinci-instruct-beta-v3", prompt=structured_input_notes, max_tokens=500)

In [41]:
print(anonymised_structured_care_notes['choices'][0]['text'])

    Customer is well in themselves this evening, made a cup tea on my arrival and put a frozen meal in the microwave and heated it to manufacturers guidelines, tonight's meal had too much rice. Customer didn't enjoy all of the rice,washed up chatted and leaving customer comfy with their cup of tea. Ahmed was also there - he is often there as he is their neighbour, and they call him ahmy. Ahmy was helpful. We then left.


### Fine Tuning

Now lets try fine tuning it with some notes

In [1]:
openai.

SyntaxError: invalid syntax (2539604145.py, line 1)